In [3]:

from modelarts.session import Session
session = Session()
session.obs.download_file(src_obs_file="obs://softcup/segmentation/panoptic_deeplab_X101_32x8d_os32_cityscapes.pth", dst_local_dir="/home/ma-user/work/configs/")

INFO:OBS:Successfully download hwc obs://softcup/segmentation/panoptic_deeplab_X101_32x8d_os32_cityscapes.pth to local /home/ma-user/work/configs/


In [7]:
import moxing as mox
mox.file.copy('s3://dsjgb/软件杯初赛视频.mp4', '/home/ma-user/work/input_video/a.mp4')

ERROR:root:Failed to call:
	func=<bound method ObsClient.getObjectMetadata of <moxing.framework.file.src.obs.client.ObsClient object at 0x7fd530c84b38>>
	args=('dsjgb', '软件杯初赛视频.mp4')
	kwargs={}
ERROR:root:
	stat:403
	errorCode:None
	errorMessage:None
	reason:Forbidden
	request-id:00000173791BA62844CA5C739C901351
	retry:0


MoxFileRespException: Error occurs when accessing to OBS.

In [2]:
from modelarts.session import Session
session = Session()
session.obs.upload_file(src_local_file='/home/ma-user/work/output/软件杯初赛视频.mp4', dst_obs_dir='obs://softcup/')

ERROR:OBS:The local file /home/ma-user/work/output/软件杯初赛视频.mp4 is not exist


Exception: The local file /home/ma-user/work/output/软件杯初赛视频.mp4 is not exist

In [6]:
import moxing as mox
mox.file.copy_parallel('/home/ma-user/work/output', 's3://dsjgd/video/ForTheWincailiao')

ERROR:root:Failed to call:
	func=<bound method ObsClient.getObjectMetadata of <moxing.framework.file.src.obs.client.ObsClient object at 0x7f5380610a20>>
	args=('dsjgd', 'video/')
	kwargs={}
ERROR:root:
	stat:403
	errorCode:None
	errorMessage:None
	reason:Forbidden
	request-id:00000173793FDA1644CA61FB840625C5
	retry:0


MoxFileRespException: Error occurs when accessing to OBS.

In [3]:
import moxing as mox
mox.file.copy_parallel('/home/ma-user/work/output', 'obs://softcup/data')

INFO:root:Using MoXing-v1.15.1-3fc51aac
INFO:root:Using OBS-Python-SDK-3.1.2


In [ ]:
from modelarts.session import Session
session = Session()
session.obs.upload_dir(src_local_dir='/home/ma-user/work/output/', dst_obs_dir='s3://dsjgd/video/ForTheWincailiao/')

In [1]:
!pip install torch===1.6.0 torchvision===0.7.0

    100% |████████████████████████████████| 748.8MB 115.1MB/s ta 0:00:014MB 110.4MB/s eta 0:00:07B 107.1MB/s eta 0:00:07                | 12.6MB 107.4MB/s eta 0:00:07.2MB/s eta 0:00:06            | 24.2MB 118.2MB/s eta 0:00:07                    | 33.2MB 38.8MB/s eta 0:00:19           | 37.5MB 63.6MB/s eta 0:00:12    | 42.8MB 86.8MB/s eta 0:00:09:00:06             | 51.4MB 115.3MB/s eta 0:00:07�▍                             | 56.1MB 141.3MB/s eta 0:00:05 |██▋                             | 60.7MB 143.1MB/s eta 0:00:05           | 65.8MB 47.2MB/s eta 0:00:15███                             | 70.7MB 142.7MB/s eta 0:00:05                           | 79.6MB 128.0MB/s eta 0:00:060:00:07��▉                            | 89.8MB 120.5MB/s eta 0:00:06 | 94.2MB 95.7MB/s eta 0:00:07|████▏                           | 98.3MB 39.3MB/s eta 0:00:17   | 102.2MB 64.7MB/s eta 0:00:10ta 0:00:11                  | 111.5MB 142.9MB/s eta 0:00:05                   | 115.8MB 67.8MB/s eta 0:00:10.8MB/s eta 0:00:11

You are using pip version 9.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install fvcore

  Found existing installation: PyYAML 3.12
    Uninstalling PyYAML-3.12:
      Successfully uninstalled PyYAML-3.12
You are using pip version 9.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install opencv-contrib-python

    100% |████████████████████████████████| 34.2MB 82.2MB/s ta 0:00:011�███████▌                    | 12.3MB 120.4MB/s eta 0:00:01██████████                 | 16.0MB 62.6MB/s eta 0:00:01MB 133.3MB/s eta 0:00:01��███████████▏  | 31.1MB 89.7MB/s eta 0:00:01
You are using pip version 9.0.1, however version 20.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from yolo import YOLO
import cv2
import time
from PIL import Image
import numpy as np
import  os

# 调用摄像头
capture=cv2.VideoCapture("input_video/软件杯决赛demo.mp4")
capture1 = cv2.VideoCapture("output/Video.avi")
fps = int(capture1.get(cv2.CAP_PROP_FPS))
size = (int(capture1.get(cv2.CAP_PROP_FRAME_WIDTH)), int(capture1.get(cv2.CAP_PROP_FRAME_HEIGHT)))
print("fps: {}\nsize: {}".format(fps, size))

# 读取视频时长（帧总数）
total = int(capture1.get(cv2.CAP_PROP_FRAME_COUNT))
print("[INFO] {} total frames in video".format(total))
# 调用VideoWrite（）函数
videoWrite = cv2.VideoWriter('output/outputVideo.avi', cv2.VideoWriter_fourcc(*'XVID'), fps, size)

diclass = {'car': 0,'bus': 0, 'person': 0, 'bike': 0, 'truck': 0, 'motor': 0,'train': 0,'rider':0,'traffic sign': 0, 'traffic light': 0}
txt = "output/totalCount.txt"  # 将要输出保存的文件地址

yolo = YOLO()
index = 0
while capture1.isOpened():
    t1 = time.time()
    # 读取某一帧
    ref, frame = capture.read()
    _, frame1 = capture1.read()
    # 进行检测
    frame,class_dict = yolo.detect_image(frame,frame1)
    with open(txt,"w") as f:
        for k in class_dict.keys():
            f.write(k + ': ' + str(class_dict[k]))  # 将字符串写入文件中
            f.write("\n")  # 换行
            
    fps = (fps + (1. / (time.time() - t1))) / 2
    #print("fps= %.2f" % (fps))
    frame = cv2.putText(frame, "fps= %.2f" % (fps), (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (144,238,144), 2)
    
    videoWrite.write(frame)
    frame = cv2.cvtColor(frame,cv2.COLOR_RGB2BGR)
    plt.figure(figsize=(10,10))
    plt.imshow(frame)
    plt.axis('on')
    plt.show()
    print(index)
    index += 1
capture.release()
capture1.release()
yolo.close_session()